### 8. Поиск аномалий (система алертов)
##### Задача: напишите систему алертов для нашего приложения
Система должна с периодичностью каждые 15 минут проверять ключевые метрики, такие как активные пользователи в ленте / мессенджере, просмотры, лайки, CTR, количество отправленных сообщений. 
1. Изучите поведение метрик и подберите наиболее подходящий метод для детектирования аномалий. 
2. В случае обнаружения аномального значения, в чат должен отправиться алерт - сообщение со следующей информацией: метрика, ее значение, величина отклонения. В сообщение можно добавить дополнительную информацию, которая поможет при исследовании причин возникновения аномалии, это может быть, например,  график, ссылки на дашборд/чарт в BI системе. 
3. Автоматизируйте систему алертов с помощью Airflow

Изучив на дашборде поведение метрик за последние 30 дней и за день, пришла к следущим выводам. <br>
Не считая некоторых точечных аномалий, которые были в рассматриваемый период. <br>
1. в целом с 1 ноября колебания метрик в большую или меньшую сторону почти не заметно: DAU_feed, likes, sent_message
2. DAU_message - есть закономерность в поведении метрики: 1 день значение метрики почти в 2 раза выше, чем значение в 2 следующих дня
3. views - закономерность в поведении метрики выявить не смогла, в разные дни она ведет себя по-разному. Но заметны в целом в течение недели колебания вверх-вниз. 
4. Колебания значений метрик в разное время суток в течение дня наблюдается у каждой метрики
##### Поэтому для DAU_message для сравнения беру метрики, которое были 3 дня назад, а для остальных метрик - средние значения за 7 последних дней и проверяю ключевые метрики с периодичностью каждые 15 минут.

In [ ]:
import numpy as np # для нахождения даты 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import telegram
import pandahouse as ph
from datetime import date, datetime, timedelta
import io # сохраняю картинки в буфере

#скрыть токен и chat_id, когда выложу в гите
import sys
import os

import math

from airflow.decorators import dag, task
#from airflow.operators.python import get_current_contex


#для подключения к clickhouse
connection = {
              'host':'https://clickhouse.lab.karpov.courses',
              'database':'simulator_20221020',
              'user':'student', 
              'password':'dpo_python_2020'
             }


# Дефолтные параметры, которые прокидываются в таски
default_args = {
                'owner': 'aksakal',
                'depends_on_past': False,
                'retries': 2,
                'retry_delay': timedelta(minutes=5),
                'start_date': datetime(2022, 11, 9), # старт дата, с которой собираем данные9/11/2022
                }

# Интервал запуска DAG
schedule_interval = '*/15 * * * *'# каждые 15 минут запускаю проверку


def check_anomaly(df, metric, a=5, n=3):
#a- это коэффициент, который будет влиять на ширину межквартильного интервала - смотрим на графике, правильно ли подобралa,
#чтобы не было часто аномалий или наоборот, чтобы не получился слишком широкий интервал
#n- коэффициент, который будет влиять на количество временных промежутков, для которых рассчитываю межквартильный интервал 
# т.е. беру несколько промежутков по 15 минут, которые имеют похожие значения метрик, для которых буду применять 
# межквартильный интервал.
    today = np.datetime64('today')


    df['q25'] = df[metric].shift(1).rolling(n).quantile(0.25)
    df['q75'] = df[metric].shift(1).rolling(n).quantile(0.75)

    # метод shift() - использую для того, чтобы на значения границ, которые буду вычислять для 15 минутки, не повлияли значения 
    # самой этой 15-минутки, для этого сдвигаю наше окно на 1 период назад
    # рассчитываю скользящее значения межквартильного размаха для метрики- для этого сначала рассчитываю 25 и 75 квантили 
    # отображать буду все значения за вчера и сегодня, чтобы график был наглядным и мб понять, как метрика вела себя в течение дня 


    # рассчитываю межквартильный размах и записываю в датафрейм
    df['iqr'] = df['q75'] - df['q25']  # межквартильный размах
    df['up'] = df['q75'] + a * df['iqr'] # верхняя граница межквартильного размаха
    df['low'] = df['q25'] - a * df['iqr'] # нижняя граница межквартильного размаха

    #чтобы границы не были рваными, еще раз сгладила границы - рассчитала границы скользящим средним
    df['up'] = df['up'].rolling(n, center=True, min_periods=1).mean() 
    df['low'] = df['low'].rolling(n, center=True, min_periods=1).mean()

    # center=True - значит текущая 15-минутка будет посередине этого окна
    # min_period=1 - задаем, чтобы границы на графике прорисовывались до конца, иначе, поскольку выбрала окно центральным 
    # (center=True) + если не хватает периодов n для последней 15-минутки, то границы не просчитываются, а проставляются None 
    # и на графике границы до конца не прорисовываются


    # прописываю условие чему равен флаг is_alert
    if df[metric].iloc[-1] < df['low'].iloc[-1] or df[metric].iloc[-1] > df['up'].iloc[-1]:
        is_alert=1
    else:
        is_alert=0

    df = df[df['date'] == today]

    return is_alert, df
    # функция будет возвращать: 
    # 1. флаг is_alert, который говорит о том, являеется ли текущее значение метрики аномальным или нет
    # 2. df -в который добавила границы межквартильного размаха





@dag(default_args=default_args, schedule_interval=schedule_interval, catchup=False)
def aksakal_my_alert():

    today = np.datetime64('today')


    # считаю средние показатели метрик за 7 дней до сегодня + ставлю дату за вчера
    @task
    def extract_feed_7day():
        query = ''' SELECT 
                            hm 
                            , avg(users_feed) as users_feed
                            , avg(views) as views
                            , avg(likes) as likes
                            , avg(ctr) as ctr
                            , date
                    FROM
                        (SELECT
                            formatDateTime(toStartOfFifteenMinutes(time), '%R') as hm
                            , uniqExact(user_id) as users_feed
                            , countIf(user_id, action = 'view') as views
                            , countIf(user_id, action = 'like') as likes
                            , likes/views as ctr
                            , today()-1 as date
                        FROM 
                            {db}.feed_actions
                        WHERE 
                            time >=  today() - 7 and time < today()

                        GROUP BY  date, hm
                        ORDER BY date, hm)

                    GROUP BY  date, hm
                    ORDER BY date, hm
                    '''
        df_cube_feed_7day = ph.read_clickhouse(query, connection=connection)
        return df_cube_feed_7day


    # считаю метрики только за сегодня
    @task
    def extract_feed_today():
        query = ''' SELECT
                        toStartOfFifteenMinutes(time) as ts
                        , toDate(time) as date
                        , formatDateTime(ts, '%R') as hm
                        , uniqExact(user_id) as users_feed
                        , countIf(user_id, action = 'view') as views
                        , countIf(user_id, action = 'like') as likes
                        , likes/views as ctr

                    FROM 
                        {db}.feed_actions
                    WHERE 
                        time >= today() and time < toStartOfFifteenMinutes(now())

                    GROUP BY date, hm, ts
                    ORDER BY date, hm 
                    '''
        df_cube_feed_today = ph.read_clickhouse(query, connection=connection)
        return df_cube_feed_today


    # WHERE time >=  today() and time < toStartOfFifteenMinutes(now()) - беру период за вчера до сегодня без текущих 15 минут,
    # тк текущая 15-минутка может быть неполная и тогда получу алерт, что данные упали, хотя это не так
    # toStartOfFifteenMinutes(now()) - округляет текущее время до начала текущей 15 минутки 
    # formatDateTime(time, '%R') as hm  - округленное до 15 минуток время выводит в формате часы-минуты 
    # добавила колонки date, hm - для удобства построения графиков 



    # считаю метрики за 3 дня до сегодня + сегодня для users_message, тк там есть закономерность каждые 3 дня (1 день рост, 2 снижение и так циклично)
    @task
    def extract_users_message():
        query = ''' SELECT 
                            ts, date, hm, users_message
                    FROM

                        (SELECT
                            toStartOfFifteenMinutes(time) as ts
                            , toDate(time) as date
                            , formatDateTime(ts, '%R') as hm
                            , uniqExact(user_id) as users_message

                        FROM 
                        {db}.message_actions
                        WHERE 
                        time >= today() and time < toStartOfFifteenMinutes(now())

                        GROUP BY date, hm, ts
                        ORDER BY date)

                        UNION ALL               

                    (SELECT
                            toStartOfFifteenMinutes(time) as ts
                            , toDate(time) as date
                            , formatDateTime(ts, '%R') as hm
                            , uniqExact(user_id) as users_message

                        FROM 
                        {db}.message_actions
                        WHERE 
                            time >= today()-3 and time < today()-2

                        GROUP BY date, hm, ts
                        ORDER BY date)
                    '''
        df_cube_users_message = ph.read_clickhouse(query, connection=connection)
        return df_cube_users_message


    # считаю средние показатели метрик за 7 дней до сегодня + ставлю дату за вчера
    # буду смотреть аномалии по алгоритму, как для метрик feed 
    @task
    def extract_sent_message():
        query = ''' 
                SELECT 
                            hm
                            , date
                            , avg(sent_message) as sent_message
                FROM

                    (SELECT
                        today()-1 as date
                        , formatDateTime(toStartOfFifteenMinutes(time), '%R') as hm
                        , count(user_id) as sent_message

                    FROM 
                    {db}.message_actions
                    WHERE 
                        time >=  today()-7 and time < today()

                    GROUP BY date, hm
                    ORDER BY date, hm)  

                GROUP BY date, hm
                ORDER BY date, hm      
                    '''
        df_cube_sent_message = ph.read_clickhouse(query, connection=connection)
        return df_cube_sent_message


    @task
    def extract_sent_message_today():
        query = ''' 
                SELECT 
                        toStartOfFifteenMinutes(time) as ts
                        , toDate(time) as date
                        , formatDateTime(ts, '%R') as hm
                        , count(user_id) as sent_message

                    FROM 
                        {db}.message_actions
                    WHERE 
                        time >= today() and time < toStartOfFifteenMinutes(now())

                    GROUP BY date, hm, ts
                    ORDER BY date, hm 
                    '''
        df_cube_sent_message_today = ph.read_clickhouse(query, connection=connection)
        return df_cube_sent_message_today


    #объединяю данные из 2 датафреймов в 1 для выявления аномалий в метриках
    @task
    def transform_data_feed(df_cube_feed_7day, df_cube_feed_today):

        df_cube_feed_7day['ts'] = df_cube_feed_7day['date'].astype ( str )+' '+df_cube_feed_7day['hm']
        df_cube_feed_7day['ts'] = pd.to_datetime(df_cube_feed_7day['ts'], format="%Y-%m-%d %H:%M")

        data_feed = pd.concat([df_cube_feed_7day, df_cube_feed_today], ignore_index=True)
        data_feed = data_feed.fillna(0)   # заменяю пропущенные значения NaN на 0  
        
        return data_feed

    @task
    def create_data_feed_dau(data_feed): 
        data_feed_dau = data_feed[['hm', 'date', 'ts', 'users_feed']].copy()
        return data_feed_dau
        
    @task
    def create_data_feed_views(data_feed): 
        data_feed_views = data_feed[['hm', 'date', 'ts', 'views']].copy()
        return data_feed_views

    @task
    def create_data_feed_likes(data_feed): 
        data_feed_likes = data_feed[['hm', 'date', 'ts', 'likes']].copy()
        return data_feed_likes

    @task
    def create_data_feed_ctr(data_feed): 
        data_feed_ctr =  data_feed[['hm', 'date', 'ts', 'ctr']].copy()
        return data_feed_ctr    


    #объединяю данные из 2 датафреймов в 1 для выявления аномалий в метриках
    @task
    def transform_data_sent_message(df_cube_sent_message, df_cube_sent_message_today):

        df_cube_sent_message['ts'] = df_cube_sent_message['date'].astype ( str )+' '+ df_cube_sent_message['hm']
        df_cube_sent_message['ts'] = pd.to_datetime(df_cube_sent_message['ts'], format="%Y-%m-%d %H:%M")

        data_sent_message = pd.concat([df_cube_sent_message, df_cube_sent_message_today], ignore_index=True)

        return data_sent_message


    # функция, которая запускает систему алертов и проверяем метрику на аномальность алгоритмом, описаным в функции check_anomaly()
    @task
    def run_alerts(df_1, metrics):
        # данные для подключения к боту
        my_token = 'token' # заменила настоящий токен, поскольку репозиторий публичный
        bot = telegram.Bot(token=my_token) # получаю доступ к бот
        chat_id = id_number # заменила настоящий id, поскольку репозиторий публичный

        df = df_1
        metric = metrics

        #для каждой метрики вызываю функцию поиска аномалий и возвращаю df и флаг, является ли текущее значениe 15минутки аномалией   
        is_alert, df = check_anomaly(df, metric) 

    # функция, которая отправляет текст и график, если текущее значение - аномалия
        if is_alert==1: 
            #!!!!!пока проверяю работу аллерта поставила, что не аллерт ==0: нужно потом поменять
            msg = '''Метрика {metric}:\n- текущее значение {current_val: .2f} \n- отклонение от предыдущего значения {last_val_diff: .2%}\nhttps://superset.lab.karpov.courses/superset/dashboard/2208/'''\
            .format(metric = metric, current_val=df[metric].iloc[-1], \
                    last_val_diff=abs(1-(df[metric].iloc[-1]/df[metric].iloc[-2]))) 

    # ссылка на дашборд с метриками за день в разрезе 15 минут, чтобы искать причину аномалий 
    # https://superset.lab.karpov.courses/superset/dashboard/2208/
    # {current_val: .2f} - округлила до 2 знаков
    # {last_val_diff: .2%} - отклонение в процентах от предыдущей 15-минутки
    # last_val_diff=abs( 1-(df[metric].iloc[-1]/df[metric].iloc[-2]) - вычисляю в %, поэтому abs(1-)
    # df[metric].iloc[-1]/df[metric].iloc[-2] - отношение значения текущей 15-минутки df[metric].iloc[-1], 
    # к значению предыдущей df[metric].iloc[-2] 


            # задаю параметр - размер изображения - на графике будет весь текущий день
            sns.set(rc={'figure.figsize': (16,10)})
            plt.tight_layout()


            #строю графики с данными за текущий день и прошлый день/среднее за 10 дней !!!!!!!!!!!
            ax = sns.lineplot(x= df['ts'], y= df[metric], label='metric')
            ax = sns.lineplot(x= df['ts'], y=df['up'], label='up')
            ax = sns.lineplot(x= df['ts'], y= df['low'], label='low')


            # разрядим подписи по оси х, чтобы был подписан не каждый тик по оси х
            # циклом задаем, как обозначать на оси х время, чтобы не каждое значение прописывалось
            for ind, label in enumerate(ax.get_xticklabels()):
                if ind % 2 == 0:
                    label.set_visible(True)
                else:
                    label.set_visible(False)

            # задаем подписи для осей
            ax.set(xlabel='time')        
            ax.set(ylabel=metric)  

            # задаем заголовок графика
            ax.set_title(metric)

            # укажим что нижняя граница оси y=0
            ax.set(ylim=(0, None))

            # отправляем график в телеграм
            plot_object = io.BytesIO()
            ax.figure.savefig(plot_object)
            plot_object.seek(0)
            plot_object.name='{0}.png'.format(metric)

            #print(msg)
            #plt.show()
            plt.close()
            
            #отправляем сообщение и графики в телеграм, если выявлена аномалия
            bot.sendMessage(chat_id=chat_id, text=msg)
            bot.sendPhoto(chat_id=chat_id, photo=plot_object)    
        return



    #вызов всех функций
    df_cube_feed_7day = extract_feed_7day()
    df_cube_feed_today = extract_feed_today()
    df_cube_users_message = extract_users_message()
    df_cube_sent_message = extract_sent_message()
    df_cube_sent_message_today = extract_sent_message_today()


    data_feed =  transform_data_feed(df_cube_feed_7day, df_cube_feed_today) 

    data_feed_dau = create_data_feed_dau(data_feed)
    data_feed_views = create_data_feed_views(data_feed)
    data_feed_likes = create_data_feed_likes(data_feed)
    data_feed_ctr = create_data_feed_ctr(data_feed)
    data_sent_message = transform_data_sent_message(df_cube_sent_message, df_cube_sent_message_today)

    run_alerts(data_feed_dau, metrics = 'users_feed')
    run_alerts(data_feed_views, metrics = 'views')
    run_alerts(data_feed_likes, metrics = 'likes')
    run_alerts(data_feed_ctr, metrics = 'ctr')
    run_alerts(df_cube_users_message, metrics = 'users_message')
    run_alerts(data_sent_message, metrics = 'sent_message')


aksakal_my_alert = aksakal_my_alert() 
